In [1]:
# Network Editing

In [2]:
import gzip
import jsonlines
import pandas as pd
import numpy as np
import json
from xml.etree.ElementTree import Element, SubElement, Comment, tostring
import xml.etree.ElementTree as ET
from pprint import pprint
import os.path
import tqdm

In [3]:
rename = {"Unnamed: 1":"Bus_Route_ID",
         "Unnamed: 2":"Buddy_Route_ID",
         "Unnamed: 3":"Travel_Time(sec)",
         "Unnamed: 4":"Daily_Tot_Rdrshp(Ind)",
         "Unnamed: 5":"Ind_route_rnk",
         "Unnamed: 6":"Buddy_route_rdrshp",
         "Unnamed: 7":"Avg_rdrshp",
         "Unnamed: 8":"Operator",
         "Unnamed: 9":"Mode",
         "Unnamed: 10":"RnkHelper",
         "Unnamed: 11":"Jt_Rnk",
         "Unnamed: 12":"Within_Op_rnk",
         "Unnamed: 22":"Summary_Head",
         "Unnamed: 23":"Summary_Values"}
excel_file = "Atlanta_toolkit_v0.5_c-7.0_b_-4.0_r5.0.xlsx"
work_sheet = "summ_hi_pt_rdrshp"
df = pd.read_excel(excel_file, sheet_name=work_sheet, index_col=0, header=1).fillna(method='ffill')
df.rename(columns=rename,inplace=True)
uOperators = df.Operator.unique()
Routes = []
top_pt_routes_pct=df.loc[10,"Summary_Values"]
# print(uOperator)
for operator in uOperators:
    df_filter = df.copy()[df.Operator == str(operator)]
    ncount = len(df_filter.index)
    df_filter = df_filter.copy()[df_filter.Within_Op_rnk <=ncount*top_pt_routes_pct]
    Routes.extend(df_filter.Bus_Route_ID.tolist())

In [4]:
%%time

# correct working 19-Jan-2021@4.30PM
# get transit routes departure and arrival times

import gzip
import jsonlines
import pandas as pd
import numpy as np
import json
import xml.etree.ElementTree as ET
from pprint import pprint
import os.path
from tqdm import tqdm


# get the path of the output_network file
network_file = 'output_network.xml.gz'
with gzip.open(network_file, 'rb') as f:
    file_content = f.read()
    network_root = ET.fromstring(file_content)
#     print(network_root.tag)
    

# get the path of the transitSchedule file
network_file = 'output_transitSchedule.xml.gz'
with gzip.open(network_file, 'rb') as f:
    file_content = f.read()
    ptschedule_root = ET.fromstring(file_content)
#     print(ptschedule_root.tag)
    
    
    
import copy
def insert_sublink_element(member):
    for links in network_root.findall("links"):
        temp = member.get("modes")
#         print(member.get("modes"))
#         temp_lst = temp.split(",")        
#         if ("artificial" in temp_lst) or ("stopFacilityLink" in temp_lst):
#             member.set("modes", "stopFacilityLink,bus,artificial")
#         elif ("bus" in temp_lst):
        if "bus" not in temp:
            member.set("modes", "bus")        
            member.set("permlanes", "1.0")
            member.set("freespeed", "20.0")
            member.set("capacity", "9999.0")
            # replace the link refId in the transitSchedule.xml
            linkRefId[member.get("id")] = (member.get("id")+"_busonly")
            # change the link Id = link + "_busonly"
            member.set("id", (member.get("id")+"_busonly"))
            links.append(member)
            # print(member.get("id"))
    
def get_linkIDelement(_id):
    member = network_root.find("./links/link[@id ='%s']"%(_id))
    txt=member.get("modes")
    try: 
        if member.get("id") not in originallinks:
            originallinks.append(member.get("id"))
            if "stopFacilityLink" not in txt or "artificial" not in txt:
                # if "artificial" not in txt: 
                if "bus" in txt:
                    member.set("modes", txt.replace("bus,",""))
                    # Create a copy
                    member2 = copy.deepcopy(member)
                    insert_sublink_element(member2)
                        # print(member.get("id")+"\n")
                    return member
    except:
        pass



# lets change things for the top PT rdrshp routes.
print("Process started")
routeId_set = Routes
stopFacilityId = {}
linkRefId = {}
counter=0
originallinks = []
mtt_stops_from = []
mtt_stops_to= []
for child in tqdm(ptschedule_root.iter('transitLine')):
    if child.get("id") in routeId_set:
        # print(child.get("id"))
        transitRoute=child.get("id")
        for subchild in child.iter("stop"):
            if subchild.get("refId") not in stopFacilityId:
                stopFacilityId[subchild.get("refId")] = subchild.get("refId") + "_STOPFACILITY_NO_" + str(counter)
                counter+=1
        for schild in child.iter("link"):
            member = get_linkIDelement(schild.get("refId"))
print("Completed transitline for loop")            
# change stopFacility@linkRefId to *_buslane
for key,value in tqdm(linkRefId.items()):
    for stopFacility_links in ptschedule_root.findall("./transitStops/stopFacility[@linkRefId ='%s']"%(str(key))):
        stopFacility_links.set("linkRefId", (value))
# change stopFacility@id to *_STOPFACILITY_COUNTER
print("Completed stopFacility@LinkRefId for loop")           
for key,value in tqdm(stopFacilityId.items()):
    for stopFacility_id in ptschedule_root.findall("./transitStops/stopFacility[@id ='%s']"%(str(key))):
        stopFacility_id.set("id", (value))
for key,value in tqdm(stopFacilityId.items()):
    for mtt_stops_from in ptschedule_root.findall("./minimalTransferTimes/relation[@fromStop ='%s']"%(str(key))):
        mtt_stops_from.set("fromStop", (value))
    for mtt_stops_to in ptschedule_root.findall("./minimalTransferTimes/relation[@toStop ='%s']"%(str(key))):
        mtt_stops_to.set("toStop", (value))
print("Completed stopFacilityId for loop")           
# change stop@refId to *_STOPFACILITY_COUNTER        
for key,value in tqdm(stopFacilityId.items()):
    for stop_refId in ptschedule_root.findall("./transitLine/transitRoute/routeProfile/stop[@refId ='%s']"%(str(key))):
        stop_refId.set("refId", (value))
# change link@refId to *_buslane
for key,value in tqdm(linkRefId.items()):
    for link_refId in ptschedule_root.findall("./transitLine/transitRoute/route/link[@refId ='%s']"%(str(key))):
        link_refId.set("refId", (value))
print("Completed Link@LinkRefId for loop")          
        
from xml.etree import ElementTree
from xml.dom import minidom

# save the modified transitSchedules as schedule.xml
# ptschedule = ptschedule_root.getroot()
# xmlstr = minidom.parseString(ET.tostring(ptschedule_root)).toprettyxml(indent="\t")
xmlstr = minidom.parseString(ET.tostring(ptschedule_root)).toprettyxml()
with open("matsim_transitSchedule.xml", "w") as f:
    f.write(xmlstr)

# save the modified matsim_network as road_network.xml
# roadnetwork = network_root.getroot()
# xmlstr = minidom.parseString(ET.tostring(network_root)).toprettyxml(indent="\t")
xmlstr = minidom.parseString(ET.tostring(network_root)).toprettyxml()
with open("matsim_network.xml", "w") as f:
    f.write(xmlstr)

0it [00:00, ?it/s]

Process started


307it [3:04:03, 35.97s/it] 
  0%|▏                                                                             | 22/13273 [00:00<02:01, 108.97it/s]

Completed transitline for loop


  1%|▉                                                                              | 26/2281 [00:00<00:18, 123.99it/s]

Completed stopFacility@LinkRefId for loop


  0%|▏                                                                                | 6/2281 [00:00<00:38, 58.52it/s]

Completed stopFacilityId for loop


100%|████████████████████████████████████████████████████████████████████████████| 13273/13273 [13:21<00:00, 16.56it/s]


Completed Link@LinkRefId for loop
Wall time: 3h 28min 21s


In [8]:
# # linkRefId.to_csv('LinkRefID.csv')
# df = pd.DataFrame.from_dict(linkRefId, orient="index")
# df.to_csv('LinkRefId.csv')

In [9]:
# # linkRefId.to_csv('LinkRefID.csv')
# df = pd.DataFrame.from_dict(stopFacilityId, orient="index")
# df.to_csv('stopFacilityId.csv')

In [10]:
# for key,value in tqdm(stopFacilityId.items()):
#     for mtt_stops_from in ptschedule_root.findall("./minimalTransferTimes/relation[@fromStop ='%s']"%(str(key))):
#         mtt_stops_from.set("fromStop", (value))
#     for mtt_stops_to in ptschedule_root.findall("./minimalTransferTimes/relation[@toStop ='%s']"%(str(key))):
#         mtt_stops_to.set("toStop", (value))
# from xml.etree import ElementTree
# from xml.dom import minidom

# # save the modified transitSchedules as schedule.xml
# # ptschedule = ptschedule_root.getroot()
# # xmlstr = minidom.parseString(ET.tostring(ptschedule_root)).toprettyxml(indent="\t")
# xmlstr = minidom.parseString(ET.tostring(ptschedule_root)).toprettyxml()
# with open("matsim_transitSchedule.xml", "w") as f:
#     f.write(xmlstr)

100%|█████████████████████████████████████████████████████████████████████████████| 2281/2281 [00:02<00:00, 863.51it/s]


In [5]:
# for key,value in linkRefId.items():
#     for links in ptschedule_root.findall("./transitLine/transitRoute/route/link[@refId ='%s']"%(str(key))):
#         links.set("refId", (value))
        
# for key,value in stopFacilityId.items():
#     for links in ptschedule_root.findall("./transitStops/stopFacility[@id ='%s']"%(str(key))):
#         links.set("id", (value))
#         if links.get("linkRefId") in linkRefId:
#             links.set("linkRefId", (linkRefId[links.get("linkRefId")]))
# #         linkRefId[links.get("linkRefId")]

# for key,value in stopFacilityId.items():
#     for links in ptschedule_root.findall("./transitLine/transitRoute/routeProfile/stop[@refId ='%s']"%(str(key))):
#         links.set("refId", (value))

        
# from xml.etree import ElementTree
# from xml.dom import minidom

# # save the modified transitSchedules as schedule.xml
# # ptschedule = ptschedule_root.getroot()
# xmlstr = minidom.parseString(ET.tostring(ptschedule_root)).toprettyxml(indent="\t")
# with open("matsim_transitSchedule.xml", "w") as f:
#     f.write(xmlstr)

# # save the modified matsim_network as road_network.xml
# # roadnetwork = network_root.getroot()
# xmlstr = minidom.parseString(ET.tostring(network_root)).toprettyxml(indent="\t")
# with open("matsim_network.xml", "w") as f:
#     f.write(xmlstr)
    
    
# for key,value in linkRefId.items():
#     for links in ptschedule_root.findall("./transitLine/transitRoute/route/link[@refId ='%s']"%(str(key))):
#         links.set("refId", (value))
        
# for key,value in stopFacilityId.items():
#     for links in ptschedule_root.findall("./transitStops/stopFacility[@id ='%s']"%(str(key))):
#         links.set("id", (value))
#         if links.get("linkRefId") in linkRefId:
#             links.set("linkRefId", (linkRefId[links.get("linkRefId")]))
# #         linkRefId[links.get("linkRefId")]

# for key,value in stopFacilityId.items():
#     for links in ptschedule_root.findall("./transitLine/transitRoute/routeProfile/stop[@refId ='%s']"%(str(key))):
#         links.set("id", (value))
        
# from xml.etree import ElementTree
# from xml.dom import minidom

# # save the modified transitSchedules as schedule.xml
# # ptschedule = ptschedule_root.getroot()
# xmlstr = minidom.parseString(ET.tostring(ptschedule_root)).toprettyxml(indent="\t")
# with open("transit_schedule.xml", "w") as f:
#     f.write(xmlstr)

# # save the modified matsim_network as road_network.xml
# # roadnetwork = network_root.getroot()
# xmlstr = minidom.parseString(ET.tostring(network_root)).toprettyxml(indent="\t")
# with open("road_network.xml", "w") as f:
#     f.write(xmlstr)       
        